In [1]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split

# Load Dataset
columns = ["target", "id", "date", "flag", "user", "text"]
data = pd.read_csv("sentiment140.csv", encoding="latin-1", names=columns)
data['sentiment'] = data['target'].map({0: 0, 2: 1, 4: 2})
data = data[['text', 'sentiment']]

# Text Cleaning
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # Remove URLs
    text = re.sub(r"\@\w+|\#", "", text)  # Remove mentions and hashtags
    text = re.sub(r"[^A-Za-z0-9\s]", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text).strip()
    return text

data['text'] = data['text'].apply(clean_text)

# Train-Test Split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['text'], data['sentiment'], test_size=0.2, random_state=42, stratify=data['sentiment']
)

In [2]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_data(texts, tokenizer):
    return tokenizer(
        list(texts),
        padding="longest",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

train_encodings = tokenize_data(train_texts, tokenizer)
test_encodings = tokenize_data(test_texts, tokenizer)


c:\Users\nives\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = SentimentDataset(train_encodings, train_labels.tolist())
test_dataset = SentimentDataset(test_encodings, test_labels.tolist())

In [4]:
from transformers import BertForSequenceClassification, AdamW, get_scheduler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3).to(device)

optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_dataset) * 3
)

c:\Users\nives\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\nives\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\nives\AppData\Local\Pro

NameError: name 'torch' is not defined

In [5]:
from torch.utils.data import DataLoader
from tqdm import tqdm

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

from torch.nn import CrossEntropyLoss

epochs = 3
gradient_accumulation_steps = 2
scaler = torch.cuda.amp.GradScaler()

loss_fn = CrossEntropyLoss()

for epoch in range(epochs):
    model.train()
    loop = tqdm(train_loader, leave=True)
    for step, batch in enumerate(loop):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.cuda.amp.autocast():
            outputs = model(**batch)
            loss = outputs.loss / gradient_accumulation_steps

        scaler.scale(loss).backward()
        
        if (step + 1) % gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            lr_scheduler.step()

        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())

  0%|          | 0/80000 [00:00<?, ?it/s]C:\Users\nives\AppData\Local\Temp\ipykernel_4152\3456639430.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 0:   0%|          | 1/80000 [00:10<238:10:35, 10.72s/it, loss=0.484]C:\Users\nives\AppData\Local\Temp\ipykernel_4152\3456639430.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 2: 100%|██████████| 80000/80000 [1:51:05<00:00, 12.00it/s, loss=0.0302]  


In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

model.eval()
predictions, true_labels = [], []

# Iterate over the test data
for batch in test_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    preds = torch.argmax(logits, dim=-1).cpu().numpy()
    labels = batch['labels'].cpu().numpy()
    predictions.extend(preds)
    true_labels.extend(labels)

# Compute and print the individual metrics
print("Accuracy:", accuracy_score(true_labels, predictions))
print("Precision:", precision_score(true_labels, predictions, average='weighted'))
print("Recall:", recall_score(true_labels, predictions, average='weighted'))
print("F1 Score:", f1_score(true_labels, predictions, average='weighted'))

# Print the detailed classification report
print("\nClassification Report:")
print(classification_report(true_labels, predictions))


NameError: name 'model' is not defined

In [3]:
model.save_pretrained("./optimized_sentiment_model")
tokenizer.save_pretrained("./optimized_sentiment_model")

NameError: name 'model' is not defined

In [5]:
from transformers import pipeline
import torch
classifier = pipeline(
    "sentiment-analysis",
    model="./optimized_sentiment_model",
    tokenizer="./optimized_sentiment_model",
    device=0 if torch.cuda.is_available() else -1
)

result = classifier("Oh Great! it's raining again")
# print(result)
label_mapping = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
final_result = {"sentiment": label_mapping[int(result[0]['label'].split('_')[-1])], "score": result[0]['score']}
print(final_result)



Device set to use cuda:0


{'sentiment': 'Negative', 'score': 0.9912443161010742}
